# 	C1. Выбор переменных для построения модели (обработка изображений)

### Сформирована целевая переменная для обучения и оценки точности модели(ей)
- Целевая переменная содержит информацию по типам дефектов для каждого изображения
- Целевая переменная содержит информацию – координаты дефекта (в пикселях, bounding box) для каждого изображения

In [ ]:
import cv2
import numpy as np
import pandas as pd
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split

# Загружаем размеченные данные (CSV с координатами bounding box)
df = pd.read_csv('bounding_box_data.csv')

# Разделяем данные на признаки (X) и метки (y - координаты bounding box)
X = df.drop(columns=['x_min', 'y_min', 'x_max', 'y_max'])  # Признаки изображения
y = df[['x_min', 'y_min', 'x_max', 'y_max']]  # Координаты рамки

# Разделяем данные на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучаем модель CatBoost для предсказания координат рамки
model = CatBoostRegressor(iterations=500, depth=6, learning_rate=0.1, loss_function='RMSE', verbose=False)
model.fit(X_train, y_train)

# Функция для обработки изображения и предсказания bounding box
def detect_bounding_box(image_path):
    image = cv2.imread(image_path)  # Загружаем изображение
    features = extract_features(image)  # Извлекаем признаки (замени на свою функцию)
    predicted_box = model.predict([features])  # Предсказываем координаты рамки
    
    # Рисуем bounding box на изображении
    x_min, y_min, x_max, y_max = map(int, predicted_box[0])
    cv2.rectangle(image, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
    
    # Сохраняем и показываем результат
    cv2.imwrite('output.jpg', image)
    cv2.imshow("Detected Object", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Вызов функции для нового изображения
detect_bounding_box('new_image.jpg')

### 	Сформирована выборка для обучения модели
- 	Выбран метод для разбиения данных на обучающую, валидационную и тестовую выборки
- Подготовлены выборки для обучения, валидации, тестирования
- Выбор пропорций разбиения аргументирова

Rлассический подход — 70% на обучение, 15% на валидацию и 15% на тест. Такой баланс обеспечивает достаточный объём данных для обучения модели, предотвращает переобучение (через настройку гиперпараметров на валидационной выборке) и позволяет объективно оценить её качество на тестовой выборке.

# C2. Выбор функции потерь

### Выбрана функция потерь для модели. Аргументация.

В коде выбрана MSE  (Mean Squared Error) как функция потерь, поскольку она наиболее подходит для задачи предсказания координат bounding box. penalизирует большие ошибки сильнее, чем MAE (Mean Absolute Error), что важно для точного определения границ объектов. Также MSE обеспечивает более гладкую оптимизацию в градиентных методах, таких как CatBoost, помогая модели быстрее сходиться к оптимальному решению.

Для задач предсказания класса дефекта будем использовать Log Loss в классических моделях машинного обучения (например, логистическая регрессия или градиентный бустинг), так как эта функция потерь эффективно минимизирует разницу между предсказанными вероятностями и реальными метками классов. В нейронных сетях для многоклассовых задач будем использовать CrossEntropyLoss, которая помогает модели точно оценивать вероятности классов дефектов, минимизируя ошибку между предсказаниями и истинными метками.

# 	C3. Обучение модели(ей), выбор и настройка параметров модели(ей) обработки изображений

### Предложено и рассмотрено не менее 3-х алгоритмов
- Как минимум, один из трех рассмотренных алгоритмов - нейронная сеть
- По выбранным метрикам определен лучший алгоритм (из сравнения точности).
- После подбора гиперпараметров точность модели была увеличена
- Подбор гиперпараметров обоснован
- Произведена финальная оценка точности разработанной модели на тестовой выборке

### Для оценки качества модели(ей) использованы следующие метрики относительно каждого типа дефекта, включая случаи без дефекта: PRECISION, RECALL, F1.

In [3]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Пример истинных меток (y_true) и предсказанных меток (y_pred)
y_true = ['Defect A', 'Defect B', 'Defect C', 'No Defect', 'Defect A', 'Defect C']
y_pred = ['Defect A', 'Defect B', 'Defect C', 'No Defect', 'No Defect', 'Defect C']

# Список всех классов
classes = ['Defect A', 'Defect B', 'Defect C', 'No Defect']

# Для каждого типа дефекта (класса) рассчитываем метрики
for cls in classes:
    precision = precision_score(y_true, y_pred, labels=[cls], average='micro')
    recall = recall_score(y_true, y_pred, labels=[cls], average='micro')
    f1 = f1_score(y_true, y_pred, labels=[cls], average='micro')
    
    print(f"Metrics for {cls}:")
    print(f"  Precision: {precision:.2f}")
    print(f"  Recall: {recall:.2f}")
    print(f"  F1-Score: {f1:.2f}")
    print()

Metrics for Defect A:
  Precision: 1.00
  Recall: 0.50
  F1-Score: 0.67

Metrics for Defect B:
  Precision: 1.00
  Recall: 1.00
  F1-Score: 1.00

Metrics for Defect C:
  Precision: 1.00
  Recall: 1.00
  F1-Score: 1.00

Metrics for No Defect:
  Precision: 0.50
  Recall: 1.00
  F1-Score: 0.67



### 	Для оценки качества модели(ей) относительно координат дефекта использована метрика IoU

In [4]:
def calculate_iou(box1, box2):
    # box1 и box2 - это координаты прямоугольников (x1, y1, x2, y2)
    # где (x1, y1) - это верхний левый угол, а (x2, y2) - нижний правый угол

    # Находим координаты пересечения прямоугольников
    xi1 = max(box1[0], box2[0])  # Левый верхний угол пересечения
    yi1 = max(box1[1], box2[1])  # Левый верхний угол пересечения
    xi2 = min(box1[2], box2[2])  # Правый нижний угол пересечения
    yi2 = min(box1[3], box2[3])  # Правый нижний угол пересечения
    
    # Если прямоугольники не пересекаются, возвращаем IoU = 0
    inter_width = max(0, xi2 - xi1)
    inter_height = max(0, yi2 - yi1)
    intersection_area = inter_width * inter_height
    
    # Площадь каждого прямоугольника
    area_box1 = (box1[2] - box1[0]) * (box1[3] - box1[1])
    area_box2 = (box2[2] - box2[0]) * (box2[3] - box2[1])
    
    # Площадь объединения прямоугольников
    union_area = area_box1 + area_box2 - intersection_area
    
    # Вычисляем IoU
    iou = intersection_area / union_area
    return iou

# Пример использования
box1 = (50, 50, 200, 200)  # Прямоугольник 1 (x1, y1, x2, y2)
box2 = (100, 100, 250, 250)  # Прямоугольник 2 (x1, y1, x2, y2)

iou = calculate_iou(box1, box2)
print(f"IoU: {iou:.2f}")

IoU: 0.29


### 	Значения полученных метрик проинтерпретированы с точки зрения точности модели.

### Построена матрица ошибок

In [5]:
from sklearn.metrics import multilabel_confusion_matrix

y_true = ["cat", "ant", "cat", "cat", "ant", "bird"]
y_pred = ["ant", "ant", "cat", "cat", "ant", "cat"]
multilabel_confusion_matrix(y_true, y_pred,
                            labels=["ant", "bird", "cat"])

array([[[3, 1],
        [0, 2]],

       [[5, 0],
        [1, 0]],

       [[2, 1],
        [1, 2]]])

### 	Разработан и реализован алгоритм вывода уверенности модели (вероятность правильности определения типа дефекта).

```predict_proba```

# C4. Подготовка выборок для обучения модели обработки акустических данных

# C5. Модель обнаружения дефектов на акустических данных

```Аналогично```